# Automated_Grading_Experiments.ipynb

Automatically grade LLM responses.

This file is experimental and output might not be used.

In [2]:
import pickle
import pandas as pd

# Load functions
from Grading_Functions import *


## Experimental GSM8k Grading

In [3]:
# Load questions and answers
import json

# Load the test question from test.jsonl
with open('GSM8k/test.jsonl', 'r') as f:
    test = f.readlines()

questions = [json.loads(t)['question'] for t in test]
answers = [json.loads(t)['answer'] for t in test]


In [4]:
print(questions[0])
print(answers[0])


Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


In [5]:
# Processed answers - keep just the last line of the answer and remove all characters except for numbers
#processed_answers = [answer.split('\n')[-1] for answer in answers]
# Use regex to remove all characters except for numbers
#import re
#processed_answers = [re.sub('[^0-9]', '', answer) for answer in processed_answers]
processed_answers = answers
# IL 2023-11-22 - want to keep answer context


In [6]:
print(processed_answers[0])


18


In [ ]:
# Load conversations
with open('Conversation_Lists.pkl', 'rb') as file:
    loaded_data = pickle.load(file)


In [ ]:
# Assign model_task_methods and conversation numbers and put conversations in a list
model_task_methods = []
conversation_numbers = []
flat_conversations_list = []
for model_task_method, conversations in loaded_data.items():
    for i, conversation in enumerate(conversations):
        model_task_methods.append(model_task_method)
        conversation_numbers.append(i+1)
        flat_conversations_list.append(conversation)


In [ ]:
# Loop over conversations
# For each conversation that has mtm containing gsm8k, 
# loop over questions until one is found that is contained in the conversation's first list element
# Get the processed answer for that question and use it to grade
correct_or_incorrect = []
for conversation_index, conversation in enumerate(flat_conversations_list):
    # Grade gsm8k items
    if 'gsm8k' in model_task_methods[conversation_index]:
        for question_index, question in enumerate(questions):
            if question in conversation[0]:
                questions_list_index = question_index
        # Feed to grading function
        correct_or_incorrect.append(grade_gsm8k_with_provided_answer(conversation, processed_answers[questions_list_index]))
    # Add NAs for non-gsm8k items
    else:
        correct_or_incorrect.append('NA')


## Output Dataset

In [ ]:
# Export model_task_method, conversation_numbers, correct_or_incorrect to Excel
# First put them in a pandas dataframe

automated_grading_df = pd.DataFrame(list(zip(model_task_methods, conversation_numbers, correct_or_incorrect)),
                                    columns = ['model_task_method', 'conversation_number', 'correct_or_incorrect'])

automated_grading_df.to_excel("Automated_Grading_Experiments.xlsx", index=False)
